In [1]:
import nest_asyncio; nest_asyncio.apply()
import stan

from data import get_data

truncated_code = """
data {
  int<lower=1> M; // number of countries
  int<lower=1> N2; // days of observed data + # of days to forecast
  int<lower=1, upper=N2> N_IMP; // number of impact invervention time effects
}
parameters {
  real<lower=0> upswing_timeeff_reduced[N_IMP,M];
  real<lower=0> sd_upswing_timeeff_reduced;
}
model {
  target += normal_lpdf(upswing_timeeff_reduced[1,:] | 0, 0.025);
  target += normal_lpdf( to_array_1d(upswing_timeeff_reduced[2:N_IMP,:]) | to_array_1d(upswing_timeeff_reduced[1:(N_IMP-1),:]), sd_upswing_timeeff_reduced);
}
"""

data = get_data()
truncated_data = {k: v for k, v in data.items() if k in ["M", "N2", "N_IMP"]}

stan_model = stan.build(truncated_code, data=truncated_data, random_seed=1)
stan_fit = stan_model.sample(num_chains=1, num_warmup=20, stepsize=1, max_depth=5, num_samples=2)
params = stan_fit.to_frame().to_dict()

Building...
Messages from stanc:
  The parameter upswing_timeeff_reduced has 2 priors.
Found model in cache. Done.
Sampling...
  0/22 [>---------------------------]   0%  1 sec/0     
 22/22 [============================] 100%  1 sec/0   Messages received during sampling:
  Gradient evaluation took 4.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.48 seconds.
  Adjust your expectations accordingly!
           three stages of adaptation as currently configured.
           Reducing each adaptation stage to 15%/75%/10% of
           the given number of warmup iterations:
             init_buffer = 3
             adapt_window = 15
             term_buffer = 2
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan_y348adue/model_c2rwpssl.stan', line 13, column 2 to column 156)
  If this warning occurs sporadic

In [2]:
import numpy as np
import numpyro; numpyro.enable_x64()
import numpyro.distributions as dist

z0 = {}
z1 = {}
for k in stan_fit.param_names:
    if k not in ["upswing_timeeff_reduced", "sd_upswing_timeeff_reduced"]:
        continue
    z0[k] = np.array([v[0] for k1, v in params.items() if k1.startswith(k)])
    z1[k] = np.array([v[1] for k1, v in params.items() if k1.startswith(k)])

    if k == "upswing_timeeff_reduced":
        shape = (data["N_IMP"], data["M"])
        z0[k] = z0[k].reshape((shape[1], shape[0])).T
        z1[k] = z1[k].reshape((shape[1], shape[0])).T
    else:
        z0[k] = z0[k].reshape(())
        z1[k] = z1[k].reshape(())
    # z0[k] = np.log(z0[k])
    # z1[k] = np.log(z1[k])
pe0 = params["lp__"][0]
pe1 = params["lp__"][1]
print("stan pe0 and pe1:", pe0, pe1)
print("stan pe diff:", pe1 - pe0)

stan pe0 and pe1: -55.268647979854 -62.03862765427405
stan pe diff: -6.76997967442005


In [3]:
npe0 = dist.Normal(z0["upswing_timeeff_reduced"][:-1], z0["sd_upswing_timeeff_reduced"]).log_prob(
    z0["upswing_timeeff_reduced"][1:]
).sum() + np.log(z0["upswing_timeeff_reduced"]).sum() + np.log(z0["sd_upswing_timeeff_reduced"]) \
    + dist.Normal(0, 0.025).log_prob(z0["upswing_timeeff_reduced"][0]).sum()
npe1 = dist.Normal(z1["upswing_timeeff_reduced"][:-1], z1["sd_upswing_timeeff_reduced"]).log_prob(
    z1["upswing_timeeff_reduced"][1:]
).sum() + np.log(z1["upswing_timeeff_reduced"]).sum() + np.log(z1["sd_upswing_timeeff_reduced"]) \
    + dist.Normal(0, 0.025).log_prob(z1["upswing_timeeff_reduced"][0]).sum()
print("numpyro pe0 and pe1:", npe0, npe1)
print("numpyro pe diff:", npe1 - npe0)

numpyro pe0 and pe1: -55.26864797985401 -62.03862765427405
numpyro pe diff: -6.769979674420043


In [4]:
npe0 = dist.TruncatedDistribution(
    dist.Normal(z0["upswing_timeeff_reduced"][:-1], z0["sd_upswing_timeeff_reduced"]), low=0.).log_prob(
    z0["upswing_timeeff_reduced"][1:]
).sum() + np.log(z0["upswing_timeeff_reduced"]).sum() + np.log(z0["sd_upswing_timeeff_reduced"]) \
    + dist.HalfNormal(0.025).log_prob(z0["upswing_timeeff_reduced"][0]).sum()
npe1 = dist.TruncatedDistribution(
    dist.Normal(z1["upswing_timeeff_reduced"][:-1], z1["sd_upswing_timeeff_reduced"]), low=0.).log_prob(
    z1["upswing_timeeff_reduced"][1:]
).sum() + np.log(z1["upswing_timeeff_reduced"]).sum() + np.log(z1["sd_upswing_timeeff_reduced"]) \
    + dist.Normal(0, 0.025).log_prob(z1["upswing_timeeff_reduced"][0]).sum()
print("numpyro pe0 and pe1:", npe0, npe1)
print("numpyro pe diff:", npe1 - npe0)

numpyro pe0 and pe1: -46.0968679263607 -54.15022184588899
numpyro pe diff: -8.053353919528291
